In [22]:

import pandas as pd
import os
import numpy as np
from pathlib import Path

file_list = os.listdir('./history')
file_list = [file for file in file_list if 'bike_usage_history' in file]

tpe_stations = pd.read_csv('TPE_bike_station.csv')
# tpe_stations = tpe_stations[['station_no',
                            #  'name_tw', 'district_tw']]


In [18]:

by_time = None

for path in file_list:
    # if '2023' not in path:
    #     continue
    df = pd.read_csv(Path('./history', path), index_col=0)
    df.columns = ['lend_time', 'lend_station_name', 'return_time',
                  'return_station_name', 'usage_time', 'source_date']
    df['source_date'] = pd.to_datetime(df['source_date'])
    df['lend_date'] = pd.to_datetime(df['lend_time']).dt.date
    df['lend_hour'] = pd.to_datetime(df['lend_time']).dt.hour
    df['return_date'] = pd.to_datetime(df['return_time']).dt.date
    df['return_hour'] = pd.to_datetime(df['return_time']).dt.hour
    df = df.drop(['lend_time', 'return_time'], axis=1)
    df['usage_time'] = pd.to_timedelta(
        df['usage_time']).dt.total_seconds().astype('int')
    df = df.merge(tpe_stations, how='inner',
                  left_on='lend_station_name', right_on='name_tw')

    df = df.merge(tpe_stations, how='inner',
                  left_on='return_station_name', right_on='name_tw')

    df = df[(df['lend_date'] == df['return_date']) & (df['lend_hour'] == df['return_hour'])]  # noqa

    df['weekend'] = pd.to_datetime(
        df['lend_date']).dt.dayofweek.isin([5, 6])
    df['weekend'] = np.where(
        df['weekend'], '假日', '平日')


    one_month_by_time = df.groupby(
        by=['lend_station_name','weekend', 'lend_hour'], as_index=False).size()

    one_month_by_time = one_month_by_time.rename(
        {
         'lend_hour': 'hour',
         'size': 'traffic_count',
         'lend_station_name':'station_name'}, axis=1)
    one_month_by_time = one_month_by_time[['station_name', 'hour',
                                           'weekend',
                                           'traffic_count'
                                           ]]

    if by_time is None:
        by_time = one_month_by_time.copy(deep=True)
    else:
        by_time = pd.concat(
            [by_time, one_month_by_time], ignore_index=True)

    print(f"finish processing: {path}")
    # break

finish processing: 2022_5_bike_usage_history.csv
finish processing: 2023_3_bike_usage_history.csv
finish processing: 2023_9_bike_usage_history.csv
finish processing: 2021_11_bike_usage_history.csv
finish processing: 2022_2_bike_usage_history.csv
finish processing: 2023_4_bike_usage_history.csv
finish processing: 2022_8_bike_usage_history.csv
finish processing: 2021_12_bike_usage_history.csv
finish processing: 2023_7_bike_usage_history.csv
finish processing: 2023_10_bike_usage_history.csv
finish processing: 2022_1_bike_usage_history.csv
finish processing: 2022_6_bike_usage_history.csv
finish processing: 2023_1_bike_usage_history.csv
finish processing: 2022_7_bike_usage_history.csv
finish processing: 2023_11_bike_usage_history.csv
finish processing: 2023_6_bike_usage_history.csv
finish processing: 2022_9_bike_usage_history.csv
finish processing: 2022_3_bike_usage_history.csv
finish processing: 2023_5_bike_usage_history.csv
finish processing: 2021_10_bike_usage_history.csv
finish processi

In [19]:
by_time.shape

(1375054, 4)

In [29]:
merged = by_time.merge(tpe_stations,how='inner',left_on='station_name',right_on='name_tw')

In [32]:
merged = merged[['station_name','district_tw','lat','lng','weekend','hour','traffic_count']]

In [33]:
merged.shape

(1375054, 7)

In [34]:
merged.to_csv('pipeline_F2_result.csv', index=False)